In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, Trainer, TrainingArguments
import pandas as pd
from tqdm.auto import tqdm
from torch.utils.data import Dataset, random_split
import torch

In [ ]:
# # load pretrained
# model_id = "roneneldan/TinyStories-1M"
# model = AutoModelForCausalLM.from_pretrained(model_id)
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# prompt = "Once upon a time there was"
# input_ids = tokenizer.encode(prompt, return_tensors="pt")
# output = model.generate(input_ids, max_length=100, num_beams=1)
# output_text = tokenizer.decode(output[0], skip_special_tokens=True)
# print(output_text)

In [ ]:
# load untrained
model_id = "roneneldan/TinyStories-1M"
config = AutoConfig.from_pretrained(model_id, local_files_only=True)
model = AutoModelForCausalLM.from_config(config)
tokenizer = AutoTokenizer.from_pretrained(model_id)
prompt = "Once upon a time there was"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.generate(input_ids, max_length=100, num_beams=1)
output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output_text)

In [ ]:
def clean_carr_ret(poem):
    return poem.replace("\r", "")
poems = pd.read_csv("data/PoetryFoundationData.csv")["Poem"].apply(clean_carr_ret)
poems.head()

In [ ]:
model.resize_token_embeddings(len(tokenizer))
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
max_length = min(max([len(tokenizer.encode(p)) for p in tqdm(poems)]), 2048)
print(f"{max_length = }")

In [ ]:
class PoemDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in tqdm(txt_list):
            encodings_dict = tokenizer(
                "<|startoftext|>" + txt + "<|endoftext|>",
                truncation=True,
                max_length=max_length,
                padding="max_length",
            )
            self.input_ids.append(torch.tensor(encodings_dict["input_ids"]))
            self.attn_masks.append(torch.tensor(encodings_dict["attention_mask"]))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
dataset = PoemDataset(poems, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

In [ ]:
for d in dataset:
    if len(d) > 10:
        print(len(d))

In [ ]:
training_args = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 10,
    logging_steps=2,
    save_steps = 4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 1,
    warmup_steps=1,
    weight_decay=0.05,
    logging_dir = "./logs",
    report_to = "none",
)

In [ ]:
Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    data_collator = lambda data: {
        "input_ids": torch.stack([f[0] for f in data]),
        "attention_mask": torch.stack([f[1] for f in data]),
        "labels": torch.stack([f[0] for f in data])
    }
).train()

In [ ]:
model = torch.load("./model.pth")
generated = tokenizer("<|startoftext|>", return_tensors="pt").input_ids

In [ ]:
sample_outputs = model.generate(
    generated,
    do_sample=True,
    top_k=5,
    max_length=50,
    top_p=0.95,
    temperature=1,
    num_return_sequences=2000,
)

for i, sample_output in enumerate(sample_outputs):
    print(f"{i}: {tokenizer.decode(sample_output, skip_special_tokens=True)}")

In [ ]:
torch.save(model, "model.pt")

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
model_id = "Helsinki-NLP/opus-mt-en-de"

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

c:\Users\gooog\anaconda3\envs\nlp_clean\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gooog\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-en-de. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [30]:
prompts = "Once upon a time, there was a little car named Beep. Beep loved to go fast and play in the sun. Beep was a healthy car because he always had good fuel. Good fuel made Beep happy and strong. One day, Beep was driving in the park when he saw a big tree. The tree had many leaves that were falling. Beep liked how the leaves fall and wanted to play with them. Beep drove under the tree and watched the leaves fall on him. He laughed and beeped his horn. Beep played with the falling leaves all day. When it was time to go home, Beep knew he needed more fuel. He went to the fuel place and got more healthy fuel. Now, Beep was ready to go fast and play again the next day. And Beep lived happily ever after."
# prompts = [s + "." for s in story.split(". ")][:-1]
# for p in prompts:
#     print(p)
print("")
print("encoding")
encodings = tokenizer(prompts, return_tensors="pt", padding=True)
# print(encodings)
print("translating")
trans_codes = model.generate(**encodings)
# print(trans_codes)
print("decoding")
translations = [tokenizer.decode(t, skip_special_tokens=True) for t in trans_codes]
print("")
for t in translations:
    print(t)


encoding
translating
decoding

Einmal gab es ein kleines Auto namens Beep. Beep liebte es, schnell zu gehen und in der Sonne zu spielen. Beep war ein gesundes Auto, weil er immer guten Treibstoff hatte. Guter Treibstoff machte Beep glücklich und stark. Eines Tages fuhr Beep im Park, als er einen großen Baum sah. Der Baum hatte viele Blätter, die fielen. Beep mochte, wie die Blätter fallen und mit ihnen spielen wollte. Beep fuhr unter dem Baum und beobachtete, wie die Blätter auf ihn fielen. Er lachte und piepste sein Horn. Beep spielte mit den fallenden Blättern den ganzen Tag. Als es Zeit war, nach Hause zu gehen, wusste Beep, dass er mehr Treibstoff brauchte. Er ging zum Kraftstoffplatz und bekam mehr gesunden Treibstoff. Nun war Beep bereit, schnell zu gehen und am nächsten Tag wieder zu spielen. Und Beep lebte glücklich bis ans Ende.
